In [1]:
from mtdScope import scopeEmulator
import matplotlib.pyplot as plt
import numpy as np
import time
#from scipy.optimize import curve_fit
#import ROOT as rt
#from jupyterthemes import jtplot


#jtplot.style()
        
#load the sample data to test
se1 = scopeEmulator()
#se1.loadData('./data/SecondRun/Jan07_205749_Run9.hdf5')
se1.loadData('./data/Run1.hdf5')
se1.sliceEvent()


KeyError: "Unable to open object (object 'waveform' doesn't exist)"

# Baseline finder in CNN model

### problem description

The idea to find baseline is that finding a region where is no obvious signal presents and do a fitting to extract the background. But on hand, the standard for "no signal presented" can be a little bit tricky to be implemented. On the other hand, the region window may change from event-to-event which will makes the implementation more tricky. However, the baseline is crutial for the integration since a wrong baseline means shifted the integration by a constant.

In [ ]:
se1.showEvent(93,[1,2,3])
points = se1.getEventAdjusted(93)

In [ ]:
points = se1.getEventAdjusted(93)
bkg, data = se1.normalized_input(points[1])
plt.plot(data)
line = np.full(1000, 0)
plt.plot(line, '--',color='red',label='Wrong baseline')
plt.legend(loc='best')

### Why CNN?
For the event show above, the baseline is found by a simple program and it is obviously higher than its background, this will affect the integration significantly especially when the noise is large.

However, finding baseline is pretty easy for a human in sense by just a glance would be able to know the baseline information. Based on this idea, I trained a CNN model to find the baseline. The training samples are a random sampled Gaussian (as signal) + random sampled flat distribution (as true background). The achitecture of this model is show below:

In [ ]:
se1.load_cnn_baseline_finder('baseline_finder_v2.h5')

### The performace of the model
With this trained model above, I'd be able to correct the baseline easily. The first figure below shows the ML corrected baseline for the event above. And the second plot is the overlay between the with and without baseline corrected signals, you can see the CNN model works well.

In [ ]:
base = se1.baseline_finder.predict(data.reshape(1,1000,1))
line = np.full(1000, base)
plt.plot(data)
plt.plot(line, '--',color='red')
print('baseline:',base)
plt.show()
n=93
chan = 1
points = se1.getEventAdjusted(n)
points2 = se1.getEventAdjusted(n)
se1.cnn_baseline_correction(points,[1,2,3])
line = np.full(se1.nsamples,0)

plt.plot(points2[chan],label='origin')
plt.plot(points[chan],label='corrected')
plt.plot(line, '--',color='red')
plt.legend(loc='lower right')
plt.show()

### The performace over all the 500 sample data
This data is got from Jongho which contained 500 events, the statistics isn't high but enough to see something interesting. 

In this case, I processed the 500 events in two ways: 
1. SetA made by the simple programer to find the baseline. This set can be benchmark for testing the ML model.
2. SetB made by using the ML baseline correction

For these two sets, I applied the same trigger condition and integrated over the channel 2 to check the distribution difference between these two methods. 

In [ ]:
se1 = scopeEmulator()
se1.loadData('./data/SecondRun/Jan07_205749_Run9.hdf5')
se1.sliceEvent()
se1.load_cnn_baseline_finder('baseline_finder_v2.h5')
charge1 = se1.charge_convertion(0,se1.nevent, 2, 0.04, [2],1)
charge0 = se1.charge_convertion(0,se1.nevent, 2, 0.04, [2],0)

This plots below shows the overlay of the distribtions of the integration from the two methods described above. One can easily see that the blue one (with the correction) have significant shape difference from the orange one (no correction). And the baseline corrected one are more close to a Landau distribution! The following plots just show the Landu fitting results.

In [ ]:
charges1 = np.multiply(charge1[1],-1.0/44000)
charges0 = np.multiply(charge0[1],-1.0/44000)
plt.hist(charges1,bins=60,alpha=0.5, label='With correction')
plt.hist(charges0,bins=60,alpha=0.5, label='No correction')
plt.legend(loc='best')
plt.xlabel('Charge (fC)')
plt.ylabel('events')
plt.show()

In [ ]:

charges1 = np.multiply(charge1[1],-1.0/4400*1e15*1e-12)
charges0 = np.multiply(charge1[1],-1.0/4400*1e15*1e-12)
hchg1= rt.TH1F("hchg1", "", 100, 0, .08)
hchg0= rt.TH1F("hchg0", "", 100, 0, .08)
for i in range(len(charges)):
    hchg1.Fill(charges[i])
    hchg0.Fill(charges[i])
#hchg1.Scale(1.0/hchg.Integral())
#hchg0.Scale(1.0/hchg.Integral())
canvas = rt.TCanvas("c", "", 500, 500)

flandau = rt.TF1("flandau", "[0]*TMath::Landau(x,[1],[2])", 0,0.08)
#bins = range(0, 0.08, 0.01)
#plt.plot(bin_edges[:-1], hist)
#plt.hist(charges, bins = 80)

#plt.xlabel("fC")


In [ ]:
flandau.SetParameters(0.1, 0.01, 0.01)

hchg.Draw()
hchg.Fit(flandau,"", "", 0,0.06)
canvas.Draw()

In [ ]:
canvas0 = rt.TCanvas("c0", "", 500, 500)
flandau.SetParameters(0.1, 0.01, 0.01)

hchg.Draw()
hchg.Fit(flandau,"", "", 0,0.06)
canvas.Draw()